# Step36, 고차 미분 이외의 용도

지금까지 한일 : 역전파 시 수행되는 계산에 대해서도 '연결'을 만들도록 했다.  
역전파의 계산 그래프를 만드는 기능 자체가 DeZero의 새로운 능력이다.  
고차 미분은 이 능력을 응용한 한 가지 예에 지나지 않는다.

**NOTE_** 새로운 DeZero에서는 역전파로 수행한 계산에 대해 또 다른 역전파할 수 있다.  
이를 double backpropagation이라고 한다.

## 36.1 double backprop의 용도 

문제 : 다음의 두 식이 주어졌을 때 x=2.0에서 x에 대한 z의 미분 $\frac{\partial z}{\partial x}$를 구하여라
$$y=x^2$$
$$z=\bigg(\frac{\partial y}{\partial x} \bigg)^3+y$$ 

In [1]:
import numpy as np 
from dezero import Variable

x = Variable(np.array(2.0))
y = x**2 
y.backward(create_graph=True)
gx = x.grad
x.cleargrad()

z = gx**3 + y 
z.backward() 
print(x.grad)

variable(100.0)


이 코드에서 중요한 부분은 y.backward(create_graph=True)이다.  
미분을 하기 위해 역전파하는 코드이다.  
이 코드가 새로운 계산 그래프를 생성한다.  

그리고 역전파가 만들어낸 계산 그래프를 사용하여 새로운 계산을 하고 다시 역전파한다.

_미분의 식을 구하고 그 식을 사용하여 계산한 다음 또 다시 미분하는 문제_를 double backprop로 해결할 수 있다.  
딥러닝 연구에서도 많이 볼 수 있는 유형이다.

**NOTE_**  
gx=x.grad는 단순한 변수(값)가 아니라 계산 그래프(식)이다.  
따라서 x.grad의 계산 그래프에 대해 추가로 역전파할 수 있다.

# 뉴턴방법과 double backprop 보충학습

- 입력이 벡터인 경우의 뉴턴 방법 
- 뉴턴 방법을 대체할 수 있는 또 다른 방법 
- double backprop의 실용적인 쓰임 예

## 다변수 함수의 뉴턴 방법 

제 3 고지에서 뉴턴 방법을 구현하면서 $y=x^4 - 2x^2$라는 수식의 최솟값을 뉴턴 방법으로 구했다.  
: '입력 변수가 하나(스칼라)인 경우의 뉴턴 방법을 구현'한 것이다.

**입력이 다차원 배열인 경우 뉴턴 방법은?**  
입력 변수를 벡터 **X**로 바꾸고 함수 $y=f(X)$  
$X=(x_1,x_2,...,x_n)$ 형태로 n개의 원소를 갖는다.

그러면 $y=f(x)$에 대한 뉴턴 방법  
$$X \leftarrow X - [\nabla^2 f(x)]^{-1} \nabla f'(x)$$

$\nabla f(x)$는 기울기를 나타낸다.  
기울기는 X의 각 원소에 대한 미분

$$\nabla f(x) = \begin{pmatrix} \frac{\partial f}{\partial x_1} \\ \frac{\partial f}{\partial x_2} \\ \vdots \\ \frac{\partial f}{\partial x_2} \end{pmatrix}$$

$\nabla^2 f(x)$는 헤세 행렬(Hessian matrix)

$$\nabla^2 f(x)} =
 \begin{pmatrix}
  \frac{\partial^2 f}{\partial x_1^2} & \frac{\partial^2 f}{\partial x_1 \partial x_2} & ... & \frac{\partial^2 f}{\partial x_1 \partial x_n} \\
  \frac{\partial^2 f}{\partial x_2 \partial x_1} & \frac{\partial^2 f}{\partial x_2^2} & ... & \frac{\partial^2 f}{\partial x_1 \partial x_n} \\
  \vdots & \vdots & ... & \vdots \\
  \frac{\partial^2 f}{\partial x_n \partial x_1} & \frac{\partial^2 f}{\partial x_n \partial x_2} & ... & \frac{\partial^2 f}{\partial x_1 \partial x_n} \\
 \end{pmatrix}

위와 같이 허세 행렬은 X의 두 원소에 대한 미분이다.  
두 원소의 조합이 이루어지기 때문에 행렬 형태로 정의된다.

기울기 $\nabla f(x)$는 $\frac{\partial f}{\partial X}$  
헤세 행렬 $\nabla^2 f(x)$는 $\frac{\partial^2 f}{\partial X \partial X^T}$

$X \leftarrow X - [\nabla^2 f(x)]^{-1} \nabla f'(x)$ 에서는 기울기와 헤세 행렬을 사용하여 X를 갱신했다.  
X를 기울기 방향으로 갱신하고  
그 진행 거리를 헤세 행렬의 역행렬을 사용하여 조정한다.  
헤세 행렬이라는 2차 미분 정보를 이용함으로써 더 공격적으로 진행할 수 있어서 목적지에 더 빠르게 도달할 수 있는것이다.

하지만 머신러닝, 신경망에서 잘 사용하지 않음

## 뉴턴 방법의 문제와 대안

머신러닝에서 뉴턴 방법의 큰 단점  
매개변수가 많아지면 헤세 행렬의 역행렬 계산에 자원이 너무 많이 소모된다.

매개변수가 n개면 n^2만큼의 메모리를 사용  
$nxn$의 역행렬 계산에는 n^3만큼 사용 

대안 : 준 뉴턴 방법(QNM:Quasi-Netwon Method)  
준 뉴턴 방법은 '헤세 행렬의 역행렬'을 근사해 사용하는 방법의 총칭(구체적인 방법이 존재하는 것은 아님)

하지만 지금까지 딥러닝 분야에서 주류는 '**기울기만을 사용한 최적화**(SGD,Momentum,Adam등)'

## double backprop의 용도: 헤세 행렬과 벡터의 곱

double backprop의 사용 예 : **헤세 행렬과 벡터의 곱**계산  
원소수가 늘어나면 헤세 행렬을 계산하는 비용이 매우 커짐.  
그러나 헤세 행렬과 벡터의 곱의 '**결과**'만 필요하다면 double backprop을 사용하여 효율적으로 구할 수 있다.



In [2]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
y = x + c
print(y)

variable([[11 22 33]
          [44 55 66]])
